**This holds the main flow of the application**

In [11]:
# includes all main imports from sub-directories
%load_ext autoreload
%autoreload 2

import os 
import sys 
import sqlite3
from pprint import pprint # text formatting 
from dotenv import load_dotenv  # from python-dotenv
import numpy as np
import pandas as pd

import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sentence_transformers import SentenceTransformer, SimilarityFunction
from openai import OpenAI
import torch

client = OpenAI(api_key=os.getenv("GPT_API_KEY"))

project_dir = os.getcwd() 
print(project_dir)

message_dir = "data/processing"
sys.path.append(message_dir)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/prestonrank/RAGMessages


## Chosen embedding approach : Asymmetric Semantic Search 

For the first MVP I will be using this instead of symmetric search because the queries themselves are not symmetrical. For an example query like "SUBJECT_NAME : When was I really annoyed at this person?" the query maps to a longer block of text containing the relevant information. 

This varies in comparison to a query like "How to learn Javascript" and finding an entry similar to "How to learn JavaScript on the web?", where this would be symmetrical. 

Pre-Trained MS MARCO Models will be used for this first implementation. 
Specifically, models tuned with normalized embeddings will be used instead of models tuned with dot products initially, normalized embeddings are more generalized, but dot products can be used as experimentation later, as they are more dynamic and may pick up additional semantic information.

SentenceTransformer.encode_query and SentenceTransformer.encode_document specifically used for encoding the corpus as well as query. 

In [12]:
large_embedding_model = "text-embedding-3-large" # dim-size :  1536  
small_embedding_model = "text-embedding-3-small" # dim-size : 3072 

bert_model_name = "multi-qa-MiniLM-L6-cos-v1" # dim-size 384
embedder = SentenceTransformer(bert_model_name, similarity_fn_name=SimilarityFunction.COSINE) # 

current_dim = 384

input = [] 
input.append("This is a test input") 
embeddings = embedder.encode(input)
# print(embeddings) 

# response = client.embeddings.create(
    # input=test_input,
   #  model=current_model
# )
#print(response.data[0].embedding)


## DATA LOADING

In [13]:
import file

subject_phone = "9365539666"
subject_name = "Paris" 
messages_per_subject = 5000

file.addToTextFile(subject_phone, messages_per_subject) # puts data into text file 

sentences_per_embedding = 6 # sentences per embedding
index_multiplier = 1 * sentences_per_embedding # looking for indexes in the text file 

batch_data = file.getTextFile(sentences_per_embedding)

corpus = [] 

sentences = ""
for batch in batch_data:
  for sentence in batch: 
    sentences += sentence + " "
  corpus.append(sentences)
  sentences = ""
print("number of text messages : " + str(len(corpus)))



number of messages from data : 4992
messages written to data txt : 4992
[NULL] values added to text file 8
number of text messages : 834


In [14]:
np_embeddings = embedder.encode_document(corpus) 

embeddings = torch.tensor(np_embeddings, dtype=torch.float32)

num_of_vectors = len(np_embeddings)
print(embeddings.shape)
print("Numbers of embedding vectors " + str(num_of_vectors))

torch.Size([834, 384])
Numbers of embedding vectors 834


In [15]:
import faiss 

index_one = faiss.IndexFlatL2(current_dim) # per subject, euclidean distance
global_index = faiss.IndexFlatL2(current_dim) # includes all of the clusters 

index_one.add(embeddings) # to add an embedding, shape : (n_vectors, current_dim)
# index.is_trained, for seeing if the index is trained 
print("Number of vectors in FAISS : " + str((index_one.ntotal)))

Number of vectors in FAISS : 834


In [6]:
query_one = "What time is the kickback?"
query_two = "all kickbacks around fall"

current_query = query_two

np_query_embedding = embedder.encode_query(current_query)

query_embedding = torch.tensor(np_query_embedding, dtype=torch.float32)
query_embedding = torch.unsqueeze(query_embedding, dim=0) # dim 1 to match the number of queries 
print(query_embedding.shape) 

print("Length of embedding " + str(len(query_embedding)))

k = 5
xq = query_embedding # shape : (n_queries, dimension)
# index.search finds the similar vectors in the FAISS DB 
D, I = index_one.search(xq, k) # I has shape : (number_of_queries, k), D has shape : (number_of_queries, k)

torch.Size([1, 384])
Length of embedding 1


In [20]:
print("Shape of Indices matrix : " + str(I.shape))
print("Shape of Distances matrix : " + str(D.shape))
print("ID of indices : " + str(I[0, :]) + "\n")
vec_ids = I[0, :].tolist()
for index in vec_ids:
    sentence = file.getTextFileLine(index, index_multiplier)
    print(sentence)
    vec = index_one.reconstruct(index)
    reconstructed_vec = torch.from_numpy(vec)
    query_tensor = torch.from_numpy(query_embedding.squeeze(0))  # Convert to tensor
    similarity_score = embedder.similarity(query_tensor.unsqueeze(0), reconstructed_vec.unsqueeze(0))
    euclidean_distance = torch.sqrt(torch.sum((query_tensor - reconstructed_vec) ** 2))
    print("similarity score : " + str(similarity_score))
    print("euclidean distance score : " + str(euclidean_distance) + "\n")

Shape of Indices matrix : (1, 5)
Shape of Distances matrix : (1, 5)
ID of indices : [102 817 257 818 166]

['[THEM] ALSO\n', '[THEM] They want to kickback on the Thursday after Halloweekend\n', '[THEM] Cause some people are leaving friday\n', '[THEM] Couldn’t tell bro, looked fried\n', '[ME] yeah the kickback friday is chill too\n', '[ME] i’m fried\n']
similarity score : tensor([[0.3572]])
euclidean distance score : tensor(1.1339)

['[ME] we were high like majority of the time 😭\n', '[ME] and drove back at 2am and got back in plano at 6am\n', '[ME] insane experience\n', '[THEM] DAMN early af for why? 😭\n', '[THEM] That’s so fun tho 🙂\u200d↕️\n', '[THEM] Alternatively Paola and I go and come back and then you join for mini kickback\n']
similarity score : tensor([[0.3245]])
euclidean distance score : tensor(1.1623)

['[THEM] Dude. I just got sprayed by a sprinkler again\n', '[THEM] Wth, it literally popped out of the ground and IMMEDIATELY hit me😭\n', '[ME] LMFAOOO\n', '[ME] you should’v

In [17]:
target_text = "They want to kickback on the Thursday after Halloweekend"
target_embedding = embedder.encode([target_text])
query_embedding = embedder.encode(["all kickbacks around fall"])
similarity = embedder.similarity(query_embedding, target_embedding)
print(f"Direct similarity: {similarity}")

Direct similarity: tensor([[0.3974]])
